In [1]:
import torch
from model.ssd import *
from utils import *
from data import *
from torch.optim import SGD
from train import train
from torch.utils.data import DataLoader

In [2]:
TOTAL_PRIORS_NUM = 9128
TOTAL_NUM_OF_CLASSES = 91

In [3]:
if torch.cuda.is_available():
#     device = torch.device('cuda:0')
    device = torch.device('cpu')
else:
    device = torch.device('cpu')

In [4]:
dataset_train_path = "/home/sarieh/coco_dataset"
dataset_val_path = "/home/sarieh/coco_dataset"
annot_train_path = "/home/sarieh/coco_dataset/annotations"

In [5]:
train_data = CocoData(dataset_train_path, annot_train_path, 
                      transform=CustomCocoTransform,
                      device = device,
                      labels_path = './data/coco/annotations/labels.txt')

loading annotations into memory...
Done (t=9.37s)
creating index...
index created!


In [6]:
val_data = CocoData(dataset_train_path, annot_train_path,
                    transform=CustomCocoTransform,
                    device = device,
                    split="validation",
                    labels_path = './data/coco/annotations/labels.txt')

loading annotations into memory...
Done (t=1.52s)
creating index...
index created!


In [7]:
criterion = MultiboxLoss(TOTAL_PRIORS_NUM, TOTAL_NUM_OF_CLASSES,threshold=0.5, alpha=1)

In [8]:
epochs = 1

In [9]:
priors = [4, 7, 6, 7, 5, 5]
channels_in = [128, 256, 512, 256, 256, 256]
loc_channel_out = [4 * prior for prior in priors]
pred_channel_out = [NUM_OF_CLASSES * prior for prior in priors]
base_model = models.resnet18(pretrained=True)
features = FeatureNet(base_model)
    
for param in base_model.parameters():
    param.requires_grad=False

In [10]:
model = PredConvNet(channels_in, loc_channel_out, pred_channel_out, features.features)

In [11]:
optimizer = SGD(model.parameters(), lr=1e-6, momentum=0.5)

In [12]:
train_loader = DataLoader(dataset=train_data, collate_fn = train_data.collate_fn ,batch_size=2)
validation_loader = DataLoader(dataset=val_data, collate_fn = val_data.collate_fn , batch_size=16)

In [13]:
train(
    model = model,
    train_loader = train_loader,
    validation_loader = validation_loader,
    optimizer = optimizer,
    criterion = criterion,
    epochs = epochs,
    device = device
)

torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])

torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])

torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])

torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])
torch.Size([2, 20, 1, 1])
loss:  nan
torch.Size([2, 16, 38, 38])
torch.Size([2, 28, 19, 19])
torch.Size([2, 24, 10, 10])
torch.Size([2, 28, 5, 5])
torch.Size([2, 20, 3, 3])

KeyboardInterrupt: 